In [1]:
import numpy as np
import pandas as pd
import geopy
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time
import matplotlib.pyplot as plt
plt.style.use('seaborn-notebook')
import csv

In [2]:
df = pd.read_csv('data/ready4model_v2.csv')

In [3]:
num_no_geo = df['business_latitude'].isnull().sum()
num_no_geo

2231

## Let's examine the addresses

In [4]:
address_no_geocoord = df.loc[df['business_latitude'].isnull(), 'business_address']

In [5]:
address_nogeo_lst = address_no_geocoord.values

In [6]:
num_vars = ['Off the Grid', 'Off The Grid', 'OFF The Grid', 'OFF THE GRID', 'Approved private locations', 'Approved Private Locations','Approved Locations']
L_nogeo = []
for address in address_nogeo_lst:
    #if address != 'Off the Grid' and address != 'Off The Grid':
    if ~(address in num_vars):
        L_nogeo.append(address)

### Clean up L_nogeo
Let's clean up L_nogeo. It has addresses like   
498 09th St Ste. C   
845 Market St #495  
Pier 45 Shed B Space 1A  
135 04th St  

In [20]:
# take care of St
L_nogeo_St = []
for street in L_nogeo:
    st2 = street.split(' St ')
    L_nogeo_St.append(st2[0])

In [22]:
# take care of Ave
L_nogeo_Ave = []
for avenue in L_nogeo_St:
    st2 = avenue.split(' Ave ')
    L_nogeo_Ave.append(st2[0])

In [23]:
# take care of Blvd
L_nogeo_Blvd = []
for blvd in L_nogeo_Ave:
    st2 = blvd.split(' Blvd ')
    L_nogeo_Blvd.append(st2[0])

In [40]:
# take care of 0 in front of a street name like in 09th St
#L_nogeo_Blvd2 = ['2449 23rd St', '498 09th', '909 Grant Ave']
L_nogeo_Zero = []
for zero in L_nogeo_Blvd:
    st2 = zero.split(' 0')
    if len(st2) < 2:
        L_nogeo_Zero.append(st2[0])
    else:
        st1 = st2[0] + ' ' + st2[1]
        L_nogeo_Zero.append(st1)

In [7]:
### Let's go thru the list and collect the list of not getting addresses

In [43]:
geolocator = Nominatim(timeout = 20)

In [47]:
location = geolocator.geocode("51 Murdell Ln Livermore CA")
print(location.address)

Murdell Lane, Livermore, Alameda County, California, 94551, United States of America


In [48]:
print(location.latitude)

37.6607282


In [11]:
# This will keep retrying the do_geocode function until it manages to return without 
# a Timeout exception being raised.
# def do_geocode(address):
#     try:
#         return geolocator.geocode(address)
#     except GeocoderTimedOut:
#         time.sleep(1)
#         return do_geocode(address)

In [49]:
# add SF CA to addresses
L_nogeo_SFCA = []
for address in L_nogeo_Zero:
    L_nogeo_SFCA.append(address + ' SF CA')

In [51]:
def do_geocode2(address):
    try:
        return geolocator.geocode(address)
    except GeocoderTimedOut:
        time.sleep(1)
        return do_geocode(address)

In [14]:
# This code takes about 40 mins to finish. It is named as ..._dropme.csv to protect geo_coords_sf.csv file
# already created. If you want to run this code again, drop "dropme" part after the run.

In [52]:
write_file = "data/geo_coords_sf_dropme.csv"
with open(write_file, "w") as output:
    for address in L_nogeo_SFCA[0:2231]:  #0:2231
        location = do_geocode2(address)
        if location != None:
            output.write( address + ',' + str(location.longitude) + ',' + str(location.latitude) +  '\n')

In [56]:
with open('data/geo_coords_sf.csv', 'r') as f:
    reader = csv.reader(f)
    counter = 0
    for row in reader:
        counter += 1
        #print(row)
print(counter)

1921
